In [4]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [5]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [6]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_NUMBER', 'des_col': 'mtr_number', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_TYPE', 'des_col': 'mtr_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_PHASE', 'des_col': 'mtr_phase', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_CAPACITY', 'des_col': 'mtr_capacity', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_CONSTANT', 'des_col': 'mtr_constant', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_DEGIT', 'des_col': 'mtr_degit', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'MTR_ELEC_TYPE', 'des_col': 'mtr_elec_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_MAKE', 'des_col': 'mtr_make', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_SEALDT', 'des_col': 'mtr_sealdt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CT_TYPE', 'des_col': 'ct_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'}, 
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},\
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REMARKS', 'des_col': 'remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_SLNO', 'des_col': 'mtr_slno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_SEALNO1', 'des_col': 'mtr_sealno1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_SEALNO2', 'des_col': 'mtr_sealno2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_SEALNO3', 'des_col': 'mtr_sealno3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_SEALNO4', 'des_col': 'mtr_sealno4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_AVAILABLE', 'des_col': 'mtr_available', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACT_CTR_N', 'des_col': 'act_ctr_n', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'ACT_CTR_D', 'des_col': 'act_ctr_d', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'ACT_PTR_N', 'des_col': 'act_ptr_n', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'ACT_PTR_D', 'des_col': 'act_ptr_d', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'MTR_VOLTAGE', 'des_col': 'mtr_voltage', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_OWNERSHIP', 'des_col': 'mtr_ownership', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OFFICE_ID', 'des_col': 'office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SBRANCH_ID', 'des_col': 'sbranch_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_DECIMAL', 'des_col': 'mtr_decimal', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'MTR_MFG_DATE', 'des_col': 'mtr_mfg_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'MF_KWH', 'des_col': 'mf_kwh', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'MF_KVA', 'des_col': 'mf_kva', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'MF_KVAH', 'des_col': 'mf_kvah', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'KWH_RDG', 'des_col': 'kwh_rdg', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'KVA_RDG', 'des_col': 'kva_rdg', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'KVAH_RDG', 'des_col': 'kvah_rdg', 'src_dtype': 'DECIMAL', 'des_dtype': 'double'},
    {'src_col': 'TOD_ID', 'des_col': 'tod_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_PHASE_NEA', 'des_col': 'mtr_phase_nea', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACCNO', 'des_col': 'accno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_AMPS', 'des_col': 'mtr_amps', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_VOLTS', 'des_col': 'mtr_volts', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'INDENTNO', 'des_col': 'indentno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_DAIL', 'des_col': 'mtr_dail', 'src_dtype': 'DECFLOAT', 'des_dtype': 'double'},
    {'src_col': 'GRNTE_PERIOD', 'des_col': 'grnte_period', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'double'},
    {'src_col': 'FORWARD_TO', 'des_col': 'forward_to', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FORWARD_FROM', 'des_col': 'forward_from', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'STR_TYPE', 'des_col': 'str_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_MTR_NO', 'des_col': 'old_mtr_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_SEALNO5', 'des_col': 'mtr_sealno5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_OWNED_BY', 'des_col': 'meter_owned_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APPROVED_BY', 'des_col': 'approved_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APPROVED_DATE', 'des_col': 'approved_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.MT_METERSTOCK', 'public', 'mt_meterstock', cols_map)
print("Total Migrated Records (mt_meterstock): " + str(migrated_row_count))


Total Migrated Records (mt_meterstock): Total Src Records: 363613 Total migrated Records to dest: 363613
